# GPU Configs

In [1]:
import os
gpu_ids = [3]

# Set CUDA_VISIBLE_DEVICES to use GPUs 1 and 3
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, gpu_ids))

import tensorflow as tf
from tensorflow.python.client import device_lib

# Check if GPU is available
print('Available devices:', device_lib.list_local_devices())
print('Is GPU available?', tf.test.is_gpu_available())

# Set memory growth to prevent OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f'{len(gpus)} GPU(s) are configured for use.')
    except RuntimeError as e:
        print(e) 


2024-12-12 21:23:55.760903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734067435.774511 1269698 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734067435.778619 1269698 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 21:23:55.796087: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Available devices: [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10909525293409662217
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 48940515328
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4813689184512951117
physical_device_desc: "device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:61:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available? True
1 GPU(s) are configured for use.


I0000 00:00:1734067438.103721 1269698 gpu_device.cc:2022] Created device /device:GPU:0 with 46673 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:61:00.0, compute capability: 8.9
I0000 00:00:1734067438.107118 1269698 gpu_device.cc:2022] Created device /device:GPU:0 with 46673 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:61:00.0, compute capability: 8.9


# Libraries

In [2]:
import tensorflow as tf
from tensorflow import *
from keras.src.utils import load_img, array_to_img
from keras.src import *
from keras.src.models import *
from keras.src.utils import *
from keras.api.metrics import AUC
import numpy as np
import pandas as pd
from tensorflow.python.ops.image_ops_impl import per_image_standardization
from keras.src.utils import file_utils
from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import csv
import os
gpu_ids = [3]

# Set CUDA_VISIBLE_DEVICES to make PyTorch see only these GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, gpu_ids))

# Constants

In [4]:
BASE_WEIGHTS_PATH = (
    "https://storage.googleapis.com/tensorflow/keras-applications/densenet/"
)

DENSENET169_WEIGHT_PATH_NO_TOP = (
    BASE_WEIGHTS_PATH
    + "densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5"
)


#body_site = "HUMERUS"

#Resize the image to 96*96

#image_size = (128,128)

#Load images as RGB, remember Image Net uses RGB!!!Not grayscale, custom CNN can use grayscale and RGB both.
color_mode = "rgb"
# color_mode = "grayscale"



# DenseNet 169 Implementation


In [7]:
def dense_block(x, blocks, name):
    """A dense block.

    Args:
        x: input tensor.
        blocks: integer, the number of building blocks.
        name: string, block label.

    Returns:
        Output tensor for the block.
    """
    for i in range(blocks):

        x = conv_block(x, 32, name=name + str(int(i)) +"_block")
    return x


def transition_block(x, reduction, name):
    """A transition block.

    Args:
        x: input tensor.
        reduction: float, compression rate at transition layers.
        name: string, block label.

    Returns:
        Output tensor for the block.
    """
    bn_axis = -1
    x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_bn"
    )(x)
    x = layers.Activation("relu", name=name + "_relu")(x)
    x = layers.Conv2D(
        int(x.shape[bn_axis] * reduction),
        1,
        use_bias=False,
        name=name + "_conv",
    )(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + "_pool")(x)
    return x


def conv_block(x, growth_rate, name):
    """A building block for a dense block.

    Args:
        x: input tensor.
        growth_rate: float, growth rate at dense layers.
        name: string, block label.

    Returns:
        Output tensor for the block.
    """
    bn_axis = -1
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_0_bn"
    )(x)
    x1 = layers.Activation("relu", name=name + "_0_relu")(x1)
    x1 = layers.Conv2D(
        4 * growth_rate, 1, use_bias=False, name=name + "_1_conv"
    )(x1)
    x1 = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name=name + "_1_bn"
    )(x1)
    x1 = layers.Activation("relu", name=name + "_1_relu")(x1)
    x1 = layers.Conv2D(
        growth_rate, 3, padding="same", use_bias=False, name=name + "_2_conv"
    )(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + "_concat")([x, x1])
    return x


def CustomizeDenseNet(
    input_shape=(96,96,3),
    weights = "imagenet"
):
    blocks = [6, 12, 32, 32]

    img_input = layers.Input(shape=input_shape)

    bn_axis = -1

    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=False, name="conv1_conv")(x)
    x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name="conv1_bn"
    )(x)
    x = layers.Activation("relu", name="conv1_relu")(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2, name="pool1")(x)

    x = dense_block(x, blocks[0], name="conv2")
    x = transition_block(x, 0.5, name="pool2")
    x = dense_block(x, blocks[1], name="conv3")
    x = transition_block(x, 0.5, name="pool3")
    x = dense_block(x, blocks[2], name="conv4")
    x = transition_block(x, 0.5, name="pool4")
    x = dense_block(x, blocks[3], name="conv5")

    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="bn")(x)
    x = layers.Activation("relu", name="relu")(x)

    inputs = img_input

    model = Functional(inputs, x, name="densenet169")

    
    if weights == "imagenet":
        weights_path = file_utils.get_file(
        "densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5",
        DENSENET169_WEIGHT_PATH_NO_TOP,
        cache_subdir="models",
        file_hash="b8c4d4c20dd625c148057b9ff1c1176b",
    )
        model.load_weights(weights_path)
    # elif weights is not None:
    #     model.load_weights(weights)


    return model


# Grad-Cams

In [50]:
#https://keras.io/examples/vision/grad_cam/
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = load_img(img_path,color_mode=color_mode,target_size=(512,512))
    img = img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    #display(Image(cam_path))

def save_gradcam_n(n, a, img_paths, imgs, model, lastconv, modelname, site="UNKNOWN"):

    for i in range(n):
        heatmap = make_gradcam_heatmap(np.expand_dims(imgs[i],axis=0), model, lastconv)
        pred = int(np.round(model.predict(np.expand_dims(imgs[i],axis=0))).flatten()[0])
        save_and_display_gradcam(img_paths[i], heatmap, cam_path="./gradcam/"+site+ '/' + modelname+"_"+str(i)+"_"+str(pred)+".jpg", alpha=a)

def find_target_layer(m):
    last_conv_layer_name = list(filter(lambda x: isinstance(x, layers.Conv2D), m.layers))[-1].name
    return last_conv_layer_name


## Grad Cam for Grayscale boosted model

In [84]:
def gray_save_gradcam_n(n, a, img_paths, imgs, model, lastconv, modelname, site="UNKNOWN"):
    for i in range(n):
        heatmap = None
        for k in range(5):
            hm = make_gradcam_heatmap(np.expand_dims(imgs[i],axis=0), model.models[k][0], lastconv[k])
            if heatmap is None:
                heatmap = 0.2 * hm
            else:
                heatmap += 0.2 * hm
        pred = int(np.round(model.predict(np.expand_dims(imgs[i],axis=0))).flatten()[0])
        save_and_display_gradcam(img_paths[i], heatmap, cam_path="./Adaboost_gradcam/"+site+ '/' + modelname+"_"+str(i)+"_"+str(pred)+".jpg", alpha=a)

def gray_find_target_layer(m):
    layer_name = []
    for j in range(5):
        last_conv_layer_name = list(filter(lambda x: isinstance(x, layers.Conv2D), boosted_model.models[j][0].layers))[-1].name
        layer_name.append(last_conv_layer_name)
    return layer_name

# Data Loader

## load data by site

In [10]:
def load_data(body_site, image_size, color_mode):
    '''
    Specify body site, image size, color mode above

    '''
    #Read in path file for training set and validation set
    train_paths = pd.read_csv('/data/home/huixian/Documents/567/MURA-v1.1/train_image_paths.csv',  header=None,names=["path"])
    val_paths = pd.read_csv('/data/home/huixian/Documents/567/MURA-v1.1/valid_image_paths.csv',header=None,names=["path"])
    #create labels to classify normal and abnormal, first intiate all to be 0
    train_paths['label']=0
    val_paths['label']=0
    #if find "positive" in path, set the label from 0 to 1
    train_paths.loc[train_paths['path'].str.contains('positive'),'label']=1
    val_paths.loc[val_paths['path'].str.contains('positive'),'label']=1

    #subset for body site images path
    train_y = train_paths[train_paths['path'].str.contains(body_site)]
    val_y = val_paths[val_paths['path'].str.contains(body_site)]

    #load training images as a 4 dimentional np array
    train_images = []
    for path in train_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        train_images.append(img_arr)
    train_X = np.array(train_images)
    print(np.shape(train_X[0]))
    #load validation images as a 4 dimentional np array
    val_images = []
    for path in val_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        val_images.append(img_arr)
    val_X = np.array(val_images)

    #set path as index (in other words, drop path column) such that only contain labels
    train_y = train_y.set_index('path')
    val_y = val_y.set_index('path')

    #calculate number of observations and percentage
    train_count_all = len(train_y)
    train_count_normal = sum(train_y['label']==0)
    train_count_abnormal = sum(train_y['label']==1)
    train_count_percentage = round(train_count_abnormal*100/train_count_all,2)
    val_count_all = len (val_y)
    val_count_normal = sum(val_y['label']==0)
    val_count_abnormal = sum(val_y['label']==1)
    val_count_percentage = round(val_count_abnormal*100/val_count_all,2)

    #print relevant information
    print(f'We are loading {body_site} {color_mode} images with {image_size[0]}*{image_size[1]} image size.')
    print(f'The training set has a total of {train_count_all} images, of them, {train_count_normal} are normal and {train_count_abnormal} are abnormal images, the percentage of abnormal is {train_count_percentage}%.')
    print(f'The validation set has a total of {val_count_all} images, of them, {val_count_normal} are normal and {val_count_abnormal} are abnormal images, the percentage of abnormal is {val_count_percentage}%.')
    print(f'The training np array X has a dimentsion of {train_X.shape}')
    print(f'The validation np array X has a dimentsion of {val_X.shape}')
    print(f'The training label y has a dimentsion of {train_y.shape}')
    print(f'The validation label y has a dimentsion of {val_y.shape}')
    return train_X, train_y, val_X, val_y

def load_val_data(body_site, image_size, color_mode):
    '''
    Specify body site, image size, color mode above

    '''
    #Read in path file for training set and validation set
    val_paths = pd.read_csv('/data/home/huixian/Documents/567/MURA-v1.1/valid_image_paths.csv',header=None,names=["path"])
    #create labels to classify normal and abnormal, first intiate all to be 0
    val_paths['label']=0
    #if find "positive" in path, set the label from 0 to 1
    val_paths.loc[val_paths['path'].str.contains('positive'),'label']=1

    #subset for body site images path
    val_y = val_paths[val_paths['path'].str.contains(body_site)]

    val_images = []
    for path in val_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        val_images.append(img_arr)
    val_X = np.array(val_images)

    val_y = val_y.set_index('path')

    return val_X, val_y



## load all data as whole

In [86]:
def load_ALL_data(image_size, color_mode):
    '''
    Specify body site, image size, color mode above

    '''
    #Read in path file for training set and validation set
    train_paths = pd.read_csv('/data/home/huixian/Documents/567/MURA-v1.1/train_image_paths.csv',  header=None,names=["path"])
    val_paths = pd.read_csv('/data/home/huixian/Documents/567/MURA-v1.1/valid_image_paths.csv',header=None,names=["path"])
    #create labels to classify normal and abnormal, first intiate all to be 0
    train_paths['label']=0
    val_paths['label']=0
    #if find "positive" in path, set the label from 0 to 1
    train_paths.loc[train_paths['path'].str.contains('positive'),'label']=1
    val_paths.loc[val_paths['path'].str.contains('positive'),'label']=1

    #subset for body site images path
    train_y = train_paths[train_paths['path'].str.contains("M")]
    val_y = val_paths[val_paths['path'].str.contains("M")]

    #load training images as a 4 dimentional np array
    train_images = []
    for path in train_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        train_images.append(img_arr)
    train_X = np.array(train_images)
    print(np.shape(train_X[0]))
    #load validation images as a 4 dimentional np array
    val_images = []
    for path in val_y['path']:
        img = load_img(path,target_size=image_size,color_mode=color_mode)
        #img_arr = per_image_standardization(img_to_array(img))
        img_arr = img_to_array(img)
        img_arr = per_image_standardization(img_arr)
        val_images.append(img_arr)
    val_X = np.array(val_images)

    #set path as index (in other words, drop path column) such that only contain labels
    train_y = train_y.set_index('path')
    val_y = val_y.set_index('path')

    #calculate number of observations and percentage
    train_count_all = len(train_y)
    train_count_normal = sum(train_y['label']==0)
    train_count_abnormal = sum(train_y['label']==1)
    train_count_percentage = round(train_count_abnormal*100/train_count_all,2)
    val_count_all = len (val_y)
    val_count_normal = sum(val_y['label']==0)
    val_count_abnormal = sum(val_y['label']==1)
    val_count_percentage = round(val_count_abnormal*100/val_count_all,2)

    #print relevant information
    print(f'*********')
    print(f'We are ALL {color_mode} images with {image_size[0]}*{image_size[1]} image size.')
    print(f'The training set has a total of {train_count_all} images, of them, {train_count_normal} are normal and {train_count_abnormal} are abnormal images, the percentage of abnormal is {train_count_percentage}%.')
    print(f'The validation set has a total of {val_count_all} images, of them, {val_count_normal} are normal and {val_count_abnormal} are abnormal images, the percentage of abnormal is {val_count_percentage}%.')
    print(f'The training np array X has a dimentsion of {train_X.shape}')
    print(f'The validation np array X has a dimentsion of {val_X.shape}')
    print(f'The training label y has a dimentsion of {train_y.shape}')
    print(f'The validation label y has a dimentsion of {val_y.shape}')
    return train_X, train_y, val_X, val_y


KeyboardInterrupt: 

In [87]:
# body_site = "FOREARM"
# body_site = "ELBOW"
# body_site = "HUMERUS"
# body_site = "WRIST"
# body_site = "HAND"
# body_site = "FINGER"
# body_site = "SHOULDER"
FOREARM_train_X, FOREARM_train_y, FOREARM_val_X, FOREARM_val_y = load_data("FOREARM",image_size,color_mode)
ELBOW_train_X, ELBOW_train_y, ELBOW_val_X, ELBOW_val_y = load_data("ELBOW",image_size,color_mode)
HUMERUS_train_X, HUMERUS_train_y, HUMERUS_val_X, HUMERUS_val_y = load_data("HUMERUS",image_size,color_mode)
WRIST_train_X, WRIST_train_y, WRIST_val_X, WRIST_val_y = load_data("WRIST",image_size,color_mode)
HAND_train_X, HAND_train_y, HAND_val_X, HAND_val_y = load_data("HAND",image_size,color_mode)
FINGER_train_X, FINGER_train_y, FINGER_val_X, FINGER_val_y = load_data("FINGER",image_size,color_mode)
SHOULDER_train_X, SHOULDER_train_y, SHOULDER_val_X, SHOULDER_val_y = load_data("SHOULDER",image_size,color_mode)

train_X, train_y, val_X, val_y = load_ALL_data(image_size,color_mode)

KeyboardInterrupt: 

# Masking

In [12]:

def mask_images(imgs,mask_size,maskn):

    imgs2 = []
    if mask_size==0 or maskn==0:
        return imgs
    else:
        num_masks = math.floor(float(imgs[0].shape[0])/mask_size)
        for img in imgs:
            xy = random.sample([(x,y) for x in range(num_masks) for y in range(num_masks)],k=maskn)
            img2 = img.copy()
            for x,y in xy:
                img2[(x*mask_size):((x+1)*mask_size),(y*mask_size):((y+1)*mask_size),:] = 0.0
            imgs2.append(img2)
        return np.array(imgs2)

# TF Cohen Kappa

In [13]:
import tensorflow as tf
import numpy as np

class CohenKappa(tf.keras.metrics.Metric):
    def __init__(self, name='cohen_kappa', num_classes=2, **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        # Initialize the confusion matrix as a variable
        self.conf_mat = self.add_weight(
            name='conf_mat',
            shape=(num_classes, num_classes),
            initializer='zeros',
            dtype=tf.float32
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to binary predictions (threshold at 0.5)
        y_pred = tf.cast(y_pred > 0.5, tf.int32)
        y_true = tf.cast(y_true, tf.int32)

        # Reshape to ensure they are 1D
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        # Compute the confusion matrix for this batch
        batch_conf_mat = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )

        # Accumulate the confusion matrix counts
        return self.conf_mat.assign_add(batch_conf_mat)

    def result(self):
        # Compute Cohen's kappa from the confusion matrix
        mat = self.conf_mat
        total = tf.reduce_sum(mat)
        # Observed Accuracy
        observed = tf.reduce_sum(tf.linalg.diag_part(mat)) / total

        # Calculate expected accuracy
        sum_rows = tf.reduce_sum(mat, axis=1)
        sum_cols = tf.reduce_sum(mat, axis=0)
        expected = tf.reduce_sum(sum_rows * sum_cols) / (total * total)

        # Cohen's kappa
        kappa = (observed - expected) / (1.0 - expected)
        return kappa

    def reset_states(self):
        # Reset the confusion matrix to zeros
        tf.keras.backend.set_value(
            self.conf_mat, 
            np.zeros((self.num_classes, self.num_classes), dtype=np.float32)
        )

# Example usage in a model:
# model.compile(optimizer='adam', loss='binary_crossentropy', 
#               metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), CohenKappa()])


# Train

In [37]:
import tensorflow as tf
import numpy as np
import random

# Fix the seed
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

image_shape = (224,224,1)

base_model = CustomizeDenseNet(weights='imagenet', input_shape=image_shape)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
# x = layers.Flatten()(x)        # Normalize the features

# A single or at most two Dense layers should usually suffice
# x = layers.Dense(1024, activation='relu')(x)  # Reduced size of the layer
# x = layers.Dropout(0.2)(x)                   # Slightly higher dropout to combat 
x = layers.Dense(1024, activation='relu')(x)  # Reduced size of the layer
x = layers.Dropout(0.1)(x)                   # Slightly higher dropout to combat overfitting

# Final classification layer
predictions = layers.Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

# print(model.summary())

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy',AUC(),CohenKappa()])
# print(model.summary())
original_datagen = tf.keras.preprocessing.image.ImageDataGenerator()  # No augmentation
original_gen = original_datagen.flow(train_X, train_y, batch_size=64, shuffle=True)
results_pre=model.fit(original_gen,validation_data=(val_X, val_y),steps_per_epoch=original_gen.n//original_gen.batch_size,validation_batch_size=64,epochs=100,verbose=1)


Epoch 1/100


/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


505/575 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.6047 - auc_5: 0.6156 - cohen_kappa: 0.1621 - loss: 0.7223

2024-12-11 13:03:22.896851: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_slice_fusion_57', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_3', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_72', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_74', 20 bytes spill stores, 20 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion_31', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion_32', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_c

575/575 ━━━━━━━━━━━━━━━━━━━━ 357s 362ms/step - accuracy: 0.6081 - auc_5: 0.6200 - cohen_kappa: 0.1688 - loss: 0.7180 - val_accuracy: 0.6071 - val_auc_5: 0.6860 - val_cohen_kappa: 0.1888 - val_loss: 0.6952
Epoch 2/100
  1/575 ━━━━━━━━━━━━━━━━━━━━ 1:31 160ms/step - accuracy: 0.6406 - auc_5: 0.6769 - cohen_kappa: 0.2756 - loss: 0.6343

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6406 - auc_5: 0.6769 - cohen_kappa: 0.2756 - loss: 0.6343 - val_accuracy: 0.5924 - val_auc_5: 0.6904 - val_cohen_kappa: 0.1567 - val_loss: 0.7194
Epoch 3/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 65s 113ms/step - accuracy: 0.6865 - auc_5: 0.7220 - cohen_kappa: 0.3307 - loss: 0.6118 - val_accuracy: 0.6653 - val_auc_5: 0.7264 - val_cohen_kappa: 0.3184 - val_loss: 0.6207
Epoch 4/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6406 - auc_5: 0.6402 - cohen_kappa: 0.2317 - loss: 0.6667 - val_accuracy: 0.6634 - val_auc_5: 0.7321 - val_cohen_kappa: 0.3144 - val_loss: 0.6167
Epoch 5/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 64s 111ms/step - accuracy: 0.7123 - auc_5: 0.7569 - cohen_kappa: 0.3849 - loss: 0.5743 - val_accuracy: 0.6390 - val_auc_5: 0.7389 - val_cohen_kappa: 0.2563 - val_loss: 0.7141
Epoch 6/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6406 - auc_5: 0.6103 - cohen_kappa: 0.1674 - loss: 0.6790 - val_accuracy: 0.6465 - val_

## additional trainings

In [49]:
model.save_weights('/data/home/huixian/Documents/567/muramodel_weights_1e-6.weights.h5')

In [54]:
model.load_weights('/data/home/huixian/Documents/567/training_log/muramodel_weights_1e-6.weights.h5')

In [48]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 30, horizontal_flip = True)

traingen = datagen.flow(train_X, train_y, batch_size = 64, shuffle = True)

model.compile(optimizer=optimizers.Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['accuracy',AUC(),CohenKappa()])

results_pre=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=original_gen.n//original_gen.batch_size,validation_batch_size=64,epochs=100,verbose=1)

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
517/575 ━━━━━━━━━━━━━━━━━━━━ 7s 125ms/step - accuracy: 0.7831 - auc_6: 0.8314 - cohen_kappa: 0.5432 - loss: 1.0283

2024-12-11 14:24:41.358394: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_slice_fusion_57', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_3', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_72', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_74', 20 bytes spill stores, 20 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion_31', 4 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_compare_reduce_fusion_32', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_c

575/575 ━━━━━━━━━━━━━━━━━━━━ 353s 371ms/step - accuracy: 0.7829 - auc_6: 0.8315 - cohen_kappa: 0.5430 - loss: 1.0245 - val_accuracy: 0.7429 - val_auc_6: 0.7939 - val_cohen_kappa: 0.4842 - val_loss: 1.3363
Epoch 2/100
  1/575 ━━━━━━━━━━━━━━━━━━━━ 1:33 162ms/step - accuracy: 0.8125 - auc_6: 0.8793 - cohen_kappa: 0.6239 - loss: 0.7542

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8125 - auc_6: 0.8793 - cohen_kappa: 0.6239 - loss: 0.7542 - val_accuracy: 0.7429 - val_auc_6: 0.7940 - val_cohen_kappa: 0.4842 - val_loss: 1.3353
Epoch 3/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 73s 128ms/step - accuracy: 0.7762 - auc_6: 0.8392 - cohen_kappa: 0.5331 - loss: 0.8937 - val_accuracy: 0.7335 - val_auc_6: 0.7921 - val_cohen_kappa: 0.4657 - val_loss: 1.2686
Epoch 4/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8125 - auc_6: 0.8621 - cohen_kappa: 0.6004 - loss: 0.6304 - val_accuracy: 0.7338 - val_auc_6: 0.7921 - val_cohen_kappa: 0.4663 - val_loss: 1.2675
Epoch 5/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 74s 128ms/step - accuracy: 0.7843 - auc_6: 0.8464 - cohen_kappa: 0.5495 - loss: 0.7946 - val_accuracy: 0.7322 - val_auc_6: 0.7898 - val_cohen_kappa: 0.4630 - val_loss: 1.2412
Epoch 6/100
575/575 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7969 - auc_6: 0.8631 - cohen_kappa: 0.5574 - loss: 0.6432 - val_accuracy: 0.7322 - val_

In [55]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 45, horizontal_flip = True)

traingen = datagen.flow(SHOULDER_train_X, SHOULDER_train_y, batch_size = 64, shuffle = True)

model.compile(optimizer=optimizers.Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['accuracy',AUC(),CohenKappa()])

results_pre=model.fit(traingen,validation_data=(SHOULDER_val_X, SHOULDER_val_y),steps_per_epoch=original_gen.n//original_gen.batch_size,validation_batch_size=64,epochs=10,verbose=1)

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
131/575 ━━━━━━━━━━━━━━━━━━━━ 6:39 899ms/step - accuracy: 0.6995 - auc_9: 0.7660 - cohen_kappa: 0.3986 - loss: 0.6380

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 282s 239ms/step - accuracy: 0.7028 - auc_9: 0.7727 - cohen_kappa: 0.4053 - loss: 0.6249 - val_accuracy: 0.6625 - val_auc_9: 0.7349 - val_cohen_kappa: 0.3264 - val_loss: 0.7692
Epoch 2/10
131/575 ━━━━━━━━━━━━━━━━━━━━ 55s 125ms/step - accuracy: 0.7159 - auc_9: 0.7837 - cohen_kappa: 0.4317 - loss: 0.6044

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.7128 - auc_9: 0.7839 - cohen_kappa: 0.4255 - loss: 0.6038 - val_accuracy: 0.6590 - val_auc_9: 0.7305 - val_cohen_kappa: 0.3195 - val_loss: 0.7781
Epoch 3/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.7240 - auc_9: 0.7959 - cohen_kappa: 0.4478 - loss: 0.5872 - val_accuracy: 0.6607 - val_auc_9: 0.7331 - val_cohen_kappa: 0.3233 - val_loss: 0.7839
Epoch 4/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.7238 - auc_9: 0.7995 - cohen_kappa: 0.4474 - loss: 0.5763 - val_accuracy: 0.6554 - val_auc_9: 0.7360 - val_cohen_kappa: 0.3127 - val_loss: 0.7876
Epoch 5/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.7316 - auc_9: 0.8066 - cohen_kappa: 0.4630 - loss: 0.5619 - val_accuracy: 0.6519 - val_auc_9: 0.7376 - val_cohen_kappa: 0.3058 - val_loss: 0.7951
Epoch 6/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - accuracy: 0.7276 - auc_9: 0.8069 - cohen_kappa: 0.4552 - loss: 0.5648 - val_accuracy: 0.6519 - val_

In [51]:
results_pre=model.fit(traingen,validation_data=(FINGER_val_X, FINGER_val_y),steps_per_epoch=original_gen.n//original_gen.batch_size,validation_batch_size=64,epochs=10,verbose=1)

Epoch 1/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7895 - auc_7: 0.8598 - cohen_kappa: 0.5480 - loss: 0.4589 - val_accuracy: 0.7223 - val_auc_7: 0.7970 - val_cohen_kappa: 0.4494 - val_loss: 0.7229
Epoch 2/10
 80/575 ━━━━━━━━━━━━━━━━━━━━ 1:04 130ms/step - accuracy: 0.7907 - auc_7: 0.8650 - cohen_kappa: 0.5527 - loss: 0.4527

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7910 - auc_7: 0.8628 - cohen_kappa: 0.5527 - loss: 0.4584 - val_accuracy: 0.7223 - val_auc_7: 0.7975 - val_cohen_kappa: 0.4494 - val_loss: 0.7195
Epoch 3/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7937 - auc_7: 0.8641 - cohen_kappa: 0.5595 - loss: 0.4558 - val_accuracy: 0.7267 - val_auc_7: 0.7973 - val_cohen_kappa: 0.4587 - val_loss: 0.7259
Epoch 4/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7902 - auc_7: 0.8615 - cohen_kappa: 0.5499 - loss: 0.4563 - val_accuracy: 0.7223 - val_auc_7: 0.7966 - val_cohen_kappa: 0.4497 - val_loss: 0.7244
Epoch 5/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7976 - auc_7: 0.8728 - cohen_kappa: 0.5689 - loss: 0.4419 - val_accuracy: 0.7245 - val_auc_7: 0.7966 - val_cohen_kappa: 0.4549 - val_loss: 0.7246
Epoch 6/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7963 - auc_7: 0.8665 - cohen_kappa: 0.5619 - loss: 0.4491 - val_accuracy: 0.7223 - val_

In [52]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 45, horizontal_flip = True)

traingen = datagen.flow(FINGER_train_X, FINGER_train_y, batch_size = 64, shuffle = True)

model.compile(optimizer=optimizers.Adam(learning_rate=1e-7), loss='binary_crossentropy', metrics=['accuracy',AUC(),CohenKappa()])

results_pre=model.fit(traingen,validation_data=(FINGER_val_X, FINGER_val_y),steps_per_epoch=original_gen.n//original_gen.batch_size,validation_batch_size=64,epochs=50,verbose=1)

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 32/575 ━━━━━━━━━━━━━━━━━━━━ 1:09 128ms/step - accuracy: 0.8161 - auc_8: 0.8962 - cohen_kappa: 0.6041 - loss: 0.3866

2024-12-11 15:51:46.129945: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_add_fusion_21', 8 bytes spill stores, 8 bytes spill loads



575/575 ━━━━━━━━━━━━━━━━━━━━ 257s 201ms/step - accuracy: 0.8012 - auc_8: 0.8751 - cohen_kappa: 0.5734 - loss: 0.4319 - val_accuracy: 0.7245 - val_auc_8: 0.7952 - val_cohen_kappa: 0.4555 - val_loss: 0.7249
Epoch 2/50
 80/575 ━━━━━━━━━━━━━━━━━━━━ 1:02 126ms/step - accuracy: 0.8042 - auc_8: 0.8785 - cohen_kappa: 0.5847 - loss: 0.4301

/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8032 - auc_8: 0.8732 - cohen_kappa: 0.5783 - loss: 0.4371 - val_accuracy: 0.7245 - val_auc_8: 0.7957 - val_cohen_kappa: 0.4552 - val_loss: 0.7234
Epoch 3/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.7918 - auc_8: 0.8672 - cohen_kappa: 0.5550 - loss: 0.4441 - val_accuracy: 0.7267 - val_auc_8: 0.7962 - val_cohen_kappa: 0.4593 - val_loss: 0.7225
Epoch 4/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7967 - auc_8: 0.8704 - cohen_kappa: 0.5632 - loss: 0.4411 - val_accuracy: 0.7267 - val_auc_8: 0.7968 - val_cohen_kappa: 0.4593 - val_loss: 0.7218
Epoch 5/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7924 - auc_8: 0.8693 - cohen_kappa: 0.5570 - loss: 0.4437 - val_accuracy: 0.7267 - val_auc_8: 0.7956 - val_cohen_kappa: 0.4593 - val_loss: 0.7236
Epoch 6/50
575/575 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.8034 - auc_8: 0.8754 - cohen_kappa: 0.5777 - loss: 0.4313 - val_accuracy: 0.7267 - val_

# BoostedNet Definition

In [34]:
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf

class BoostedDenseNet:
    def __init__(self, n_estimators=5, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.weights = None
        
    def create_model(self, input_shape):
        """Creates a single DenseNet model with the same architecture"""
        base_model = CustomizeDenseNet(input_shape=input_shape, weights = 'xx')
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        predictions = layers.Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        model.load_weights('/data/home/huixian/Documents/567/training_log/muramodel_weights.weights.h5')
        model.compile(optimizer=optimizers.Adam(learning_rate=1e-6),
                     loss='binary_crossentropy',
                     metrics=['accuracy', AUC(), CohenKappa()])
        return model
        
    def fit(self, X, y, validation_data=None, epochs=10):
        """Fits multiple DenseNet models using boosting"""
        n_samples = len(X)
        self.weights = np.ones(n_samples) / n_samples
        
        for i in range(self.n_estimators):
            print(f"Training model {i+1}/{self.n_estimators}")
            
            # Create and train a new model
            model = self.create_model(input_shape=X.shape[1:])
            
            # Create weighted dataset
            sample_weights = self.weights * n_samples
            
            # Train the model
            datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=45,
                horizontal_flip=True
            )
            
            traingen = datagen.flow(
                X, y,
                sample_weight=sample_weights,
                batch_size=64,
                shuffle=True
            )
            
            model.fit(
                traingen,
                validation_data=validation_data,
                steps_per_epoch=len(X)//64,
                epochs=epochs,
                verbose=1
            )
            
            # Make predictions
            predictions = (model.predict(X) > 0.5).astype(int)
            
            # Calculate error and model weight
            incorrect = (predictions.flatten() != y.flatten())
            error = np.sum(self.weights * incorrect) / np.sum(self.weights)
            
            # Avoid division by zero and log(0)
            error = np.clip(error, 1e-10, 1-1e-10)
            model_weight = self.learning_rate * np.log((1 - error) / error)
            
            # Update sample weights
            self.weights *= np.exp(model_weight * incorrect)
            self.weights /= np.sum(self.weights)  # Normalize weights
            
            # Save the model and its weight
            self.models.append((model, model_weight))
            
    def predict(self, X):
        """Makes predictions using weighted voting of all models"""
        predictions = np.zeros(len(X))
        total_weight = 0
        
        for model, model_weight in self.models:
            predictions += model_weight * model.predict(X).flatten()
            total_weight += model_weight
            
        predictions /= total_weight
        return (predictions > 0.5).astype(int)
    
    def predict_proba(self, X):
        """Returns probability predictions"""
        predictions = np.zeros(len(X))
        total_weight = 0
        
        for model, model_weight in self.models:
            predictions += model_weight * model.predict(X).flatten()
            total_weight += model_weight
            
        return predictions / total_weight

# BoostedNet Trainer

In [35]:
import tensorflow as tf 
import numpy as np
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix
import pandas as pd
import csv
import time
from datetime import datetime

def train_boosted_model(train_X, train_y, val_X, val_y, 
                       n_estimators=3, 
                       learning_rate=0.1,
                       epochs_per_model=10,
                       save_prefix="boosted_model"):
    """
    Train a boosted DenseNet model and save results
    """
    # Initialize model
    print("Initializing boosted model...")
    boosted_model = BoostedDenseNet(n_estimators=n_estimators, 
                                   learning_rate=learning_rate)
    
    # Record start time
    start_time = time.time()
    
    # Train model
    print(f"Starting training with {n_estimators} estimators...")
    history = boosted_model.fit(
        train_X, 
        train_y,
        validation_data=(val_X, val_y),
        epochs=epochs_per_model
    )
    
    # Calculate training time
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")

    # Make predictions
    print("Generating predictions...")
    val_preds = boosted_model.predict(val_X)
    val_probs = boosted_model.predict_proba(val_X)

    # Calculate metrics
    print("Calculating metrics...")
    metrics = {
        'auc': roc_auc_score(val_y, val_probs),
        'kappa': cohen_kappa_score(val_y, val_preds),
        'conf_matrix': confusion_matrix(val_y, val_preds)
    }
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save metrics
    metrics_df = pd.DataFrame({
        'timestamp': [timestamp],
        'n_estimators': [n_estimators],
        'learning_rate': [learning_rate],
        'epochs_per_model': [epochs_per_model],
        'training_time': [training_time],
        'auc': [metrics['auc']],
        'kappa': [metrics['kappa']],
        'tn': [metrics['conf_matrix'][0,0]],
        'fp': [metrics['conf_matrix'][0,1]],
        'fn': [metrics['conf_matrix'][1,0]],
        'tp': [metrics['conf_matrix'][1,1]]
    })
    
    metrics_file = f"{save_prefix}_metrics_{timestamp}.csv"
    metrics_df.to_csv(metrics_file, index=False)
    print(f"Saved metrics to {metrics_file}")
    
    # Save ROC curve data
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(val_y, val_probs)
    roc_df = pd.DataFrame({
        'fpr': fpr,
        'tpr': tpr
    })
    
    roc_file = f"{save_prefix}_roc_{timestamp}.csv"
    roc_df.to_csv(roc_file, index=False)
    print(f"Saved ROC curve data to {roc_file}")
    
    # Save individual model weights
    for i, (model, weight) in enumerate(boosted_model.models):
        weight_file = f"{save_prefix}_model{i+1}_{timestamp}.weights.h5"
        model.save_weights(weight_file)
        print(f"Saved weights for model {i+1} to {weight_file}")
    
    # Print summary
    print("\nTraining Summary:")
    print(f"Number of estimators: {n_estimators}")
    print(f"Learning rate: {learning_rate}")
    print(f"Epochs per model: {epochs_per_model}")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"Validation AUC: {metrics['auc']:.4f}")
    print(f"Validation Kappa: {metrics['kappa']:.4f}")
    print("\nConfusion Matrix:")
    print(metrics['conf_matrix'])
    
    return boosted_model, metrics



Loading data...
Loaded weights for model 1 from densenet_boosted_model1_20241211_174317.weights.h5


/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1026 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loaded weights for model 2 from densenet_boosted_model2_20241211_174317.weights.h5
Loaded weights for model 3 from densenet_boosted_model3_20241211_174317.weights.h5
Loaded weights for model 4 from densenet_boosted_model4_20241211_174317.weights.h5
Loaded weights for model 5 from densenet_boosted_model5_20241211_174317.weights.h5


## BoostedNet Train/Load code

In [63]:
# Set random seeds for reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

# Training parameters
PARAMS = {
    'n_estimators': 5,
    'learning_rate': 1e-6,
    'epochs_per_model': 10
}

# Load and preprocess data
print("Loading data...")
image_size = (224, 224)
color_mode = "grayscale"

# Train on all body sites
# train_X, train_y, val_X, val_y = load_ALL_data(image_size, color_mode)

# Train model
# print("\nStarting boosted model training...")
# boosted_model, metrics = train_boosted_model(
#     train_X, train_y['label'].values, 
#     val_X, val_y['label'].values,
#     **PARAMS,
#     save_prefix="densenet_boosted"
# )

def load_boosted_model_weights(timestamp, n_estimators=5, learning_rate=1e-6):
    """
    Load weights for all models in the boosted ensemble
    """
    # Initialize a new BoostedDenseNet instance
    boosted_model = BoostedDenseNet(n_estimators=n_estimators, learning_rate=learning_rate)
    boosted_model.models = []
    
    for i in range(n_estimators):
        # Create a new model using the create_model method from BoostedDenseNet
        model = boosted_model.create_model(input_shape=(224,224,1))
        
        # Load the saved weights
        weight_file = f"densenet_boosted_model{i+1}_{timestamp}.weights.h5"
        model.load_weights(weight_file)
        
        # For now, use equal weights for all models since we don't save the model weights
        # You might want to save and load these weights separately in your training code
        model_weight = 1.0 / n_estimators
        
        # Add to models list as tuple of (model, weight)
        boosted_model.models.append((model, model_weight))
        print(f"Loaded weights for model {i+1} from {weight_file}")
    
    return boosted_model

timestamp = "20241211_174317"
boosted_model = load_boosted_model_weights(
    timestamp=timestamp, 
    n_estimators=5,
    learning_rate=1e-6
)
# Example usage:
# timestamp = "20241211_174317"  # Replace with your actual timestamp
# boosted_model = load_boosted_model_weights(timestamp, n_estimators=5)


Loading data...
Loaded weights for model 1 from densenet_boosted_model1_20241211_174317.weights.h5


/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 1026 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loaded weights for model 2 from densenet_boosted_model2_20241211_174317.weights.h5
Loaded weights for model 3 from densenet_boosted_model3_20241211_174317.weights.h5
Loaded weights for model 4 from densenet_boosted_model4_20241211_174317.weights.h5
Loaded weights for model 5 from densenet_boosted_model5_20241211_174317.weights.h5


# BoostedNet Eval Code

In [36]:
# Optional: Evaluate on individual body sites
if True:  # Set to True to evaluate on individual sites
    print("\nEvaluating on individual body sites...")
    site_metrics = []
    
    for site in ["WRIST", "SHOULDER", "HUMERUS", "HAND", "FOREARM", "FINGER", "ELBOW"]:
        print(f"\nEvaluating {site}...")
        site_val_X, site_val_y = load_val_data(site, image_size, color_mode)
        
        # Get predictions
        site_preds = boosted_model.predict(site_val_X)
        site_probs = boosted_model.predict_proba(site_val_X)
        
        # Calculate metrics
        site_metrics.append({
            'site': site,
            'auc': roc_auc_score(site_val_y['label'], site_probs),
            'kappa': cohen_kappa_score(site_val_y['label'], site_preds)
        })
    
    # Save site-specific metrics
    site_metrics_df = pd.DataFrame(site_metrics)
    site_metrics_file = f"densenet_boosted_site_metrics_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    site_metrics_df.to_csv(site_metrics_file, index=False)
    print(f"\nSaved site-specific metrics to {site_metrics_file}")


Evaluating on individual body sites...

Evaluating WRIST...
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 465ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 458ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 419ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 415ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 418ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Evaluating SHOULDER...
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

Evaluating HUMERUS...
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
9/9 ━━

# ImageNet Eval Code

In [59]:
import tensorflow as tf 
import numpy as np
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix
import pandas as pd
import csv
import time
from datetime import datetime
def evaluate_sites(model, image_size=(224,224), color_mode="rgb"):
    """
    Evaluate model performance on individual body sites
    """
    print("\nEvaluating on individual body sites...")
    site_metrics = []
    
    # Save ROC curve data for each site
    roc_curves = {}
    
    for site in ["WRIST", "SHOULDER", "HUMERUS", "HAND", "FOREARM", "FINGER", "ELBOW"]:
        print(f"\nEvaluating {site}...")
        site_val_X, site_val_y = load_val_data(site, image_size, color_mode)
        
        # Get predictions
        site_probs = model.predict(site_val_X)
        site_preds = (site_probs > 0.5).astype(int)
        
        # Calculate metrics
        conf_matrix = confusion_matrix(site_val_y, site_preds)
        
        site_metrics.append({
            'site': site,
            'auc': roc_auc_score(site_val_y, site_probs),
            'kappa': cohen_kappa_score(site_val_y, site_preds),
            'accuracy': accuracy_score(site_val_y, site_preds),
            'tn': conf_matrix[0,0],
            'fp': conf_matrix[0,1],
            'fn': conf_matrix[1,0],
            'tp': conf_matrix[1,1]
        })
        
        # Calculate ROC curve
        fpr, tpr, _ = roc_curve(site_val_y, site_probs)
        roc_curves[site] = {'fpr': fpr, 'tpr': tpr}
    
    # Save metrics
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save site-specific metrics
    site_metrics_df = pd.DataFrame(site_metrics)
    site_metrics_file = f"densenet_site_metrics_{timestamp}.csv"
    site_metrics_df.to_csv(site_metrics_file, index=False)
    print(f"\nSaved site-specific metrics to {site_metrics_file}")
    
    # Save ROC curves for each site
    for site, curve_data in roc_curves.items():
        roc_df = pd.DataFrame({
            'fpr': curve_data['fpr'],
            'tpr': curve_data['tpr']
        })
        roc_file = f"densenet_roc_{site}_{timestamp}.csv"
        roc_df.to_csv(roc_file, index=False)
        print(f"Saved ROC curve data for {site} to {roc_file}")
        
    # Print summary
    print("\nSite-specific Results:")
    print(site_metrics_df.to_string(index=False))
    
    return site_metrics_df, roc_curves

import tensorflow as tf
import numpy as np
import random

# Fix the seed
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

image_shape = (224,224,3)

base_model = CustomizeDenseNet(weights='imagenet', input_shape=image_shape)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
# x = layers.Flatten()(x)        # Normalize the features

# A single or at most two Dense layers should usually suffice
# x = layers.Dense(1024, activation='relu')(x)  # Reduced size of the layer
# x = layers.Dropout(0.2)(x)                   # Slightly higher dropout to combat 
x = layers.Dense(1024, activation='relu')(x)  # Reduced size of the layer
x = layers.Dropout(0.1)(x)                   # Slightly higher dropout to combat overfitting

# Final classification layer
predictions = layers.Dense(1, activation='sigmoid')(x)


model = Model(inputs=base_model.input, outputs=predictions)
# Train your model first
# model.compile(optimizer=optimizers.Adam(learning_rate=1e-6), 
#              loss='binary_crossentropy', 
#              metrics=['accuracy', AUC(), CohenKappa()])

# Training code here...

# After training, evaluate on individual sites
# site_metrics, roc_curves = evaluate_sites(model)

# Gradcam Save Code

In [ ]:
save_gradcam_n(20,0.5,FOREARM_train_y.index[FOREARM_train_y['label']==1].to_numpy(),FOREARM_train_X[np.where(FOREARM_train_y["label"] == 1)[0]],model,find_target_layer(model),"DN169_train_pos", site = 'test')

In [85]:
from tqdm import tqdm

train_X_list = [FOREARM_train_X, ELBOW_train_X, HUMERUS_train_X, WRIST_train_X, HAND_train_X, FINGER_train_X, SHOULDER_train_X]
train_y_list = [FOREARM_train_y, ELBOW_train_y, HUMERUS_train_y, WRIST_train_y, HAND_train_y, FINGER_train_y, SHOULDER_train_y]
val_X_list = [FOREARM_val_X, ELBOW_val_X, HUMERUS_val_X, WRIST_val_X, HAND_val_X, FINGER_val_X, SHOULDER_val_X]
val_y_list = [FOREARM_val_y, ELBOW_val_y, HUMERUS_val_y, WRIST_val_y, HAND_val_y, FINGER_val_y, SHOULDER_val_y]
sites = ["FOREARM", "ELBOW", "HUMERUS", "WRIST", "HAND", "FINGER", "SHOULDER"]

for i in tqdm(range(len(train_X_list))):
    gray_save_gradcam_n(20,0.5,train_y_list[i].index[train_y_list[i]['label']==1].to_numpy(),train_X_list[i][np.where(train_y_list[i]["label"] == 1)[0]],boosted_model,gray_find_target_layer(boosted_model),"DN169_train_pos", site = sites[i])
    gray_save_gradcam_n(20,0.5,train_y_list[i].index[train_y_list[i]['label']==0].to_numpy(),train_X_list[i][np.where(train_y_list[i]["label"] == 0)[0]],boosted_model,gray_find_target_layer(boosted_model),"DN169_train_neg", site = sites[i])
    gray_save_gradcam_n(20,0.5,val_y_list[i].index[val_y_list[i]['label']==1].to_numpy(),val_X_list[i][np.where(val_y_list[i]["label"] == 1)[0]],boosted_model,gray_find_target_layer(boosted_model),"DN169_val_pos", site = sites[i])
    gray_save_gradcam_n(20,0.5,val_y_list[i].index[val_y_list[i]['label']==0].to_numpy(),val_X_list[i][np.where(val_y_list[i]["label"] == 0)[0]],boosted_model,gray_find_target_layer(boosted_model),"DN169_val_neg", site = sites[i])


  0%|          | 0/7 [00:00<?, ?it/s]/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_16165']]
Received: inputs=Tensor(shape=(1, 224, 224, 1))
  warnings.warn(msg)
/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_16765']]
Received: inputs=Tensor(shape=(1, 224, 224, 1))
  warnings.warn(msg)
/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_17365']]
Received: inputs=Tensor(shape=(1, 224, 224, 1))
  warnings.warn(msg)
/data/home/huixian/.conda/envs/scratch/lib/python3.12/site-packages/keras/src/models/functional.py:238: User

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


/tmp/ipykernel_1269698/3378587414.py:46: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  jet = cm.get_cmap("jet")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━

/tmp/ipykernel_1269698/3378587414.py:43: RuntimeWarning: invalid value encountered in cast
  heatmap = np.uint8(255 * heatmap)


...

 71%|███████▏  | 5/7 [25:52<10:20, 310.37s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━

 86%|████████▌ | 6/7 [31:02<05:10, 310.10s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━

100%|██████████| 7/7 [36:11<00:00, 310.28s/it]


# Legacy Codes

In [ ]:


# for layer in model.layers[:595]:
#     layer.trainable = True
# for layer in model.layers[595:]:
#     layer.trainable = False

# for i, layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=30,horizontal_flip=True)
# traingen = datagen.flow(train_X, train_y, batch_size=32,shuffle=True)

# model.compile(optimizer=optimizers.Adam(learning_rate=1e-7), loss='binary_crossentropy', metrics=['accuracy',AUC()])

# results_post=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=1)

# for layer in model.layers[:595]:
#     layer.trainable = False
# for layer in model.layers[595:]:
#     layer.trainable = True

# for i, layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=30,horizontal_flip=True)
# traingen = datagen.flow(train_X, train_y, batch_size=32,shuffle=True)

# model.compile(optimizer=optimizers.Adam(learning_rate=1e-7), loss='binary_crossentropy', metrics=['accuracy',AUC()])

# results_post=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=32,epochs=10,verbose=1)

# for layer in base_model.layers:
#     layer.trainable = False

# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=60,horizontal_flip=True)
# traingen = datagen.flow(train_X, train_y, batch_size=16,shuffle=True)

# opt = optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',AUC()])
# # print(model.summary())
# results_pre=model.fit(traingen,validation_data=(val_X, val_y),steps_per_epoch=traingen.n//traingen.batch_size,validation_batch_size=16,epochs=10,verbose=1)

In [ ]:
# from sklearn.metrics import cohen_kappa_score
# # After training, get predictions for the validation set
# val_preds_prob = model.predict(val_X)  # shape: (num_samples, 1)
# val_preds = (val_preds_prob > 0.5).astype(np.int32).ravel()  # convert probabilities to binary predictions

# # Compute Cohen's kappa
# kappa = cohen_kappa_score(val_y, val_preds)
# print("Cohen's kappa on the validation set:", kappa)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Cohen's kappa on the validation set: np.float64(0.3687969507456612)


In [ ]:
# for layer in model.layers[:595]:
#     layer.trainable = False
# for layer in model.layers[595:]:
#     layer.trainable = True

# for i, layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

# # for i, layer in enumerate(model.layers):
# #     print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv20_block_0_bn False
8 conv20_block_0_relu False
9 conv20_block_1_conv False
10 conv20_block_1_bn False
11 conv20_block_1_relu False
12 conv20_block_2_conv False
13 conv20_block_concat False
14 conv21_block_0_bn False
15 conv21_block_0_relu False
16 conv21_block_1_conv False
17 conv21_block_1_bn False
18 conv21_block_1_relu False
19 conv21_block_2_conv False
20 conv21_block_concat False
21 conv22_block_0_bn False
22 conv22_block_0_relu False
23 conv22_block_1_conv False
24 conv22_block_1_bn False
25 conv22_block_1_relu False
26 conv22_block_2_conv False
27 conv22_block_concat False
28 conv23_block_0_bn False
29 conv23_block_0_relu False
30 conv23_block_1_conv False
31 conv23_block_1_bn False
32 conv23_block_1_relu False
33 conv23_block_2_conv False
34 conv23_block_concat False
35 conv24_block_0_bn False
36 conv24_block_0_relu False
37 conv24_bloc

In [ ]:
# import random

# results_pre_df = pd.DataFrame(results_pre.history)
# # results_post_df = pd.DataFrame(results_post.history)
# results_df = results_pre_df  #.append(results_post_df,sort=False)

# with open('DN169_globalavgpool_2dense1024_aug_metrics.csv', 'w') as f:
#     results_df.to_csv(f)

# fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
# fpr = np.array(fpr)
# tpr = np.array(tpr)
# fpr_tpr = np.stack((fpr,tpr),axis=1)
# header=['fpr','tpr']
# with open('DN169_globalavgpool_2dense1024_aug_AUC.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(header)
#     writer.writerows(fpr_tpr)

# kappa_conf=[]

# for n in range(17):
#     print(kappa_conf)
#     kappa_conf_temp=[n]
#     val_X_mask=mask_images(val_X,24,n)
#     kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X_mask),y_truth=val_y))
#     conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X_mask)))
#     kappa_conf_temp.extend(conf_temp[0])
#     kappa_conf_temp.extend(conf_temp[1])
#     kappa_conf.append(kappa_conf_temp)

# header=['masking','kappa','tn','fp','fn','tp']
# with open('DN169_globalavgpool_2dense1024_aug_kappa_conf.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(header)
#     writer.writerows(kappa_conf)

# print(find_target_layer(model))


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[]
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0,
  np.float64(0.36879695074566116),
  np.int64(176),
  np.int64(38),
  np.int64(110),
  np.int64(137)]]


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:

from sklearn.metrics import accuracy_score

kappa_conf=[]

# for site in ["WRIST","SHOULDER","HUMERUS","HAND","FOREARM","FINGER","ELBOW"]:
for site in ["FINGER"]:
    val_X, val_y = load_val_data(site,image_size,color_mode)
    kappa_conf_temp=[site]
    kappa_conf_temp.append(accuracy_score(val_y,np.round(model.predict(val_X))))
    kappa_conf_temp.append(cohen_kappa(y_prob=model.predict(val_X),y_truth=val_y))
    conf_temp=confusion_matrix(val_y,np.round(model.predict(val_X)))
    kappa_conf_temp.extend(conf_temp[0])
    kappa_conf_temp.extend(conf_temp[1])
    kappa_conf.append(kappa_conf_temp)
    fpr, tpr, threshold = roc_curve(val_y,model.predict(val_X))
    fpr = np.array(fpr)
    tpr = np.array(tpr)
    fpr_tpr = np.stack((fpr,tpr),axis=1)
    header=['fpr','tpr']
    with open('DN169_globalavgpool_2dense1024_aug_AUC_'+site+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(fpr_tpr)

header=['site','accuracy','kappa','tn','fp','fn','tp']
with open('DN169_globalavgpool_2dense1024_aug_metrics_sites.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(kappa_conf)